In [1]:
# CREATE CASSANDRA CONNECTION
import cassandra
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [2]:
# CREATE A KEYSPACE TO DO OUR WORK
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

# SET THE KEYSPACE/SCHEMA NAME
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

Imagine we would like to start creating a new Music Library of albums.
We want to ask 4 question of our data

1. Give me every album in my music library that was released in a given year
    - `select * from music_library WHERE YEAR=1970`
2. Give me the album that is in my music library that was released in a given year by "The Beatles"
    - `select * from music_library WHERE YEAR = 1970 AND ARTIST_NAME = "The Beatles"`
3. Give me all the albums released in a given year in a give location
    - `select * from music_library WHERE YEAR = 1970 AND LOCATION = "Liverpool"`
4. Give me city that the albumn "Let It Be" was recorded
    - `select city from music_library WHERE YEAR = "1970" AND ARTIST_NAME = "The Beatles" AND ALBUM_NAME="Let it Be"`
- How should we model this data? What should be our Primary Key and Partition Key? Since our data is looking for the YEAR let's start with that. 
    - From there we will add clustering columns on Artist Name and Album Name.
    - Table Name: music_library
        - column 1: Year
        - column 2: Artist Name
        - column 3: Album Name
        - column 4: City
        - PRIMARY KEY(year, artist_name, album_name)


In [3]:
# CREATE THE TABLE
query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(year int, artist_name text, album_name text, city text, PRIMARY KEY (year, artist_name, album_name))"
try:
 session.execute(query)
except Exception as e:
 print(e)

In [4]:
# INSERT DATA INTO TABLE
query = "INSERT INTO music_library (year, artist_name, album_name, city)"
query = query + " VALUES (%s, %s, %s, %s)"

try:
 session.execute(query, (1970, "The Beatles", "Let it Be", "Liverpool"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1965, "The Beatles", "Rubber Soul", "Oxford"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1965, "The Who", "My Generation", "London"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1966, "The Monkees", "The Monkees", "Los Angeles"))
except Exception as e:
 print(e)

try:
 session.execute(query, (1970, "The Carpenters", "Close To You", "San Diego"))
except Exception as e:
 print(e)

In [5]:
# QUERY ONE
query = "select * from music_library WHERE YEAR=1970"
try:
 rows = session.execute(query)
except Exception as e:
 print(e)

for row in rows:
 print (row.year, row.artist_name, row.album_name, row.city)

1970 The Beatles Let it Be Liverpool
1970 The Carpenters Close To You San Diego


In [6]:
# QUERY TWO
query = "select * from music_library WHERE YEAR=1970 AND ARTIST_NAME='The Beatles'"
try:
 rows = session.execute(query)
except Exception as e:
 print(e)

for row in rows:
 print (row.year, row.artist_name, row.album_name, row.city)

1970 The Beatles Let it Be Liverpool


In [9]:
# QUERY THREE
query = "select * from music_library WHERE YEAR=1970 AND ARTIST_NAME='The Beatles' AND CITY = 'Liverpool' ALLOW FILTERING"
try:
 rows = session.execute(query)
except Exception as e:
 print(e)

for row in rows:
 print (row.year, row.artist_name, row.album_name, row.city)

1970 The Beatles Let it Be Liverpool


In [10]:
# QUERY THREE
query = "select * from music_library WHERE YEAR=1970 AND ARTIST_NAME='The Beatles' AND ALBUM_NAME='Let it Be'"
try:
 rows = session.execute(query)
except Exception as e:
 print(e)

for row in rows:
 print (row.year, row.artist_name, row.album_name, row.city)

1970 The Beatles Let it Be Liverpool


In [11]:
# FOR DEMO PURPOSES DROP THE TABLE
try:
 session.execute('Drop table music_library')
except Exception as e:
 print(e)

In [12]:
# CLOSE THE SESSION AND CLUSTER CONNECTION
session.shutdown()
cluster.shutdown()